## download dataset from http://yann.lecun.com/exdb/mnist/

In [33]:
'''
import requests
import os
from tqdm import tqdm  # for progress bar

def download_file(url, destination):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 KB
    progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)
    
    with open(destination, 'wb') as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()
    
    if total_size != 0 and progress_bar.n != total_size:
        print("Error: Failed to download the file.")
        os.remove(destination)  # Remove partially downloaded file
    else:
        print("Download complete.")

# URLs for the MNIST dataset files
base_url = "http://yann.lecun.com/exdb/mnist/"
files = [
    "train-images-idx3-ubyte.gz",
    "train-labels-idx1-ubyte.gz",
    "t10k-images-idx3-ubyte.gz",
    "t10k-labels-idx1-ubyte.gz"
]

# Destination directory for downloaded files
destination_dir = "Downloads"

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Download each file
for file in files:
    file_url = base_url + file
    file_destination = os.path.join(destination_dir, file)
    download_file(file_url, file_destination)
'''


'\nimport requests\nimport os\nfrom tqdm import tqdm  # for progress bar\n\ndef download_file(url, destination):\n    response = requests.get(url, stream=True)\n    total_size = int(response.headers.get(\'content-length\', 0))\n    block_size = 1024  # 1 KB\n    progress_bar = tqdm(total=total_size, unit=\'iB\', unit_scale=True)\n    \n    with open(destination, \'wb\') as file:\n        for data in response.iter_content(block_size):\n            progress_bar.update(len(data))\n            file.write(data)\n    progress_bar.close()\n    \n    if total_size != 0 and progress_bar.n != total_size:\n        print("Error: Failed to download the file.")\n        os.remove(destination)  # Remove partially downloaded file\n    else:\n        print("Download complete.")\n\n# URLs for the MNIST dataset files\nbase_url = "http://yann.lecun.com/exdb/mnist/"\nfiles = [\n    "train-images-idx3-ubyte.gz",\n    "train-labels-idx1-ubyte.gz",\n    "t10k-images-idx3-ubyte.gz",\n    "t10k-labels-idx1-ubyt

In [34]:
import tensorflow as tf 


In [35]:
tf.compat.v1.disable_eager_execution()


In [38]:
#tf.compat.v1.flags.DEFINE_integer("is_train", 1, "indicate whether to train the model or use data to predict")
#FLAGS = tf.compat.v1.flags.FLAGS

In [87]:
import tensorflow as tf
import numpy as np
import gzip



tf.compat.v1.disable_eager_execution()

def read_idx_file(file_path):
    with gzip.open(file_path, 'rb') as f:
        # Read the magic number
        magic = int.from_bytes(f.read(4), 'big')
        
        # Read the number of images
        num_images = int.from_bytes(f.read(4), 'big')
        
        # Read the number of rows and columns per image
        rows = int.from_bytes(f.read(4), 'big')
        cols = int.from_bytes(f.read(4), 'big')
        
        # Read the image data
        image_data = np.frombuffer(f.read(), dtype=np.uint8)
        image_data = image_data.reshape(num_images, rows * cols)  # Flatten images to 1D array
        
        return image_data

def read_idx_labels(file_path):
    with gzip.open(file_path, 'rb') as f:
        # Read the magic number
        magic = int.from_bytes(f.read(4), 'big')
        
        # Read the number of labels
        num_labels = int.from_bytes(f.read(4), 'big')
        
        # Read the label data
        label_data = np.frombuffer(f.read(), dtype=np.uint8)
        
        return label_data

def full_connected_nn(train_images, train_labels, test_images, test_labels):
    
    with tf.compat.v1.variable_scope("data"):
        
        X = tf.compat.v1.placeholder(tf.float32, [None, 784], name="feature")
        y_true = tf.compat.v1.placeholder(tf.float32, [None, 10], name="label")

    with tf.compat.v1.variable_scope("hidden"):
        
        # the first layer randomization for the initialization of parameters 
        weight_hidden = tf.Variable(tf.random.normal([784, 64], mean=0.0, stddev=1.0), name="weightes_hidden")
        bias_hidden = tf.Variable(tf.random.normal([64], mean=0.0, stddev=1.0), name="biases_hidden")
        
        a1 = tf.matmul(X, weight_hidden) + bias_hidden
        
    with tf.compat.v1.variable_scope("fc"):
        # the second layer randomization for the initialization of parameters 
        weight_fc = tf.Variable(tf.random.normal([64, 10], mean=0.0, stddev=1.0), name="weightes_fc")
        bias_fc = tf.Variable(tf.random.normal([10], mean=0.0, stddev=1.0), name="biases_fc")
        
        # fully connected operations 
        # predict ten labels 
        y_predict = tf.matmul(a1, weight_fc) + bias_fc

    with tf.compat.v1.variable_scope("softmax_cross"):
        
        # compute the loss, cross entropy 
        all_loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_predict, name="compute_loss")
        
        # compute the average 
        loss = tf.reduce_mean(all_loss)

        
    with tf.compat.v1.variable_scope("GD"):
        
        # gradient decent 
        train_op = tf.compat.v1.train.GradientDescentOptimizer(0.1).minimize(loss)

        
    with tf.compat.v1.variable_scope("accuracy"):
        
        equal_list = tf.equal(tf.argmax(y_predict, 1), tf.argmax(y_true, 1))
        
        accuracy = tf.reduce_mean(tf.cast(equal_list, tf.float32))

    #tf.summary.scalar("loss", loss)
    #tf.summary.scalar("acc", accuracy)

    #tf.summary.histogram("weight_hidden", weight_hidden)
    #tf.summary.histogram("bias_hidden", bias_hidden)
    #tf.summary.histogram("weight_fc", weight_fc)
    #tf.summary.histogram("bias_fc", bias_fc)
    
    # merge the results 
    #merged = tf.compat.v1.summary.merge_all()
    
    saver = tf.compat.v1.train.Saver()

    with tf.compat.v1.Session() as sess:
        
        sess.run(tf.compat.v1.global_variables_initializer())
        
        # get the local model 
        ckpt = tf.compat.v1.train.latest_checkpoint("./tmp/model/")
        print(ckpt)
        
        # check if the model exists 
        #if ckpt:
        #    saver.restore(sess, ckpt)

        
        for i in range(2000):

            #file_writer = tf.compat.v1.summary.FileWriter("./tmp/summary/", graph=sess.graph)
            # create events files 
            # batch size of 50
            #mnist_x, mnist_y = train_images.next_batch(50)

            mnist_x = train_images[i*50%train_images.shape[0]:(i*50+50)%train_images.shape[0]]
            mnist_y = train_labels[i*50%train_labels.shape[0]:(i*50+50)%train_labels.shape[0]]

            loss_run, acc, _ = sess.run([loss,accuracy, train_op], 
                              feed_dict={X: mnist_x, y_true: mnist_y})

            print(" %d the iteration and the 50 sample loss is %f, accuracy is %f " % (i, loss_run, acc))

            #file_writer.add_summary(summary, i)

            if i % 100 == 0:
                saver.save(sess, "./tmp/model/fc_nn_model")


        for i in range(100):
            # use 100 test sample to predict 

            #image, label = test_images.next_batch(1)
            image = test_images[i:i+1]
            label = test_labels[i:i+1]

            print(" the d% sample, real pic number is %d, neural network predicted number is %d" % (
                i+1,
                tf.argmax(sess.run(y_true, feed_dict={X: image, y_true: label}), 1).eval(),
                tf.argmax(sess.run(y_predict, feed_dict={X: image, y_true: label}), 1).eval()
            ))


# Assuming you have the MNIST dataset files in the specified paths
train_images = read_idx_file("Downloads/train-images-idx3-ubyte.gz")
train_labels = read_idx_labels("Downloads/train-labels-idx1-ubyte.gz")

test_images = read_idx_file("Downloads/t10k-images-idx3-ubyte.gz")
test_labels = read_idx_labels("Downloads/t10k-labels-idx1-ubyte.gz")

# Preprocess the data (normalize pixel values)
train_images = train_images / 255.0
test_images = test_images / 255.0

# Convert labels to one-hot encoding
train_labels_one_hot = tf.keras.utils.to_categorical(train_labels, 10)
test_labels_one_hot = tf.keras.utils.to_categorical(test_labels, 10)

# Run the neural network
full_connected_nn(train_images, train_labels_one_hot, test_images, test_labels_one_hot)

None
 0 the iteration and the 50 sample loss is 97.428665, accuracy is 0.120000 
 1 the iteration and the 50 sample loss is 61.971333, accuracy is 0.180000 
 2 the iteration and the 50 sample loss is 66.817863, accuracy is 0.300000 
 3 the iteration and the 50 sample loss is 45.936813, accuracy is 0.380000 
 4 the iteration and the 50 sample loss is 46.396240, accuracy is 0.280000 
 5 the iteration and the 50 sample loss is 47.395420, accuracy is 0.420000 
 6 the iteration and the 50 sample loss is 33.837955, accuracy is 0.440000 
 7 the iteration and the 50 sample loss is 25.067879, accuracy is 0.440000 
 8 the iteration and the 50 sample loss is 29.079819, accuracy is 0.500000 
 9 the iteration and the 50 sample loss is 22.548735, accuracy is 0.560000 
 10 the iteration and the 50 sample loss is 24.832548, accuracy is 0.500000 
 11 the iteration and the 50 sample loss is 25.011616, accuracy is 0.460000 
 12 the iteration and the 50 sample loss is 21.800755, accuracy is 0.480000 
 13 

 201 the iteration and the 50 sample loss is 2.760341, accuracy is 0.860000 
 202 the iteration and the 50 sample loss is 3.867591, accuracy is 0.880000 
 203 the iteration and the 50 sample loss is 2.421162, accuracy is 0.860000 
 204 the iteration and the 50 sample loss is 6.622053, accuracy is 0.780000 
 205 the iteration and the 50 sample loss is 7.383390, accuracy is 0.840000 
 206 the iteration and the 50 sample loss is 4.675188, accuracy is 0.840000 
 207 the iteration and the 50 sample loss is 2.967821, accuracy is 0.880000 
 208 the iteration and the 50 sample loss is 5.010060, accuracy is 0.780000 
 209 the iteration and the 50 sample loss is 1.157489, accuracy is 0.900000 
 210 the iteration and the 50 sample loss is 1.636846, accuracy is 0.940000 
 211 the iteration and the 50 sample loss is 1.147528, accuracy is 0.920000 
 212 the iteration and the 50 sample loss is 0.518369, accuracy is 0.920000 
 213 the iteration and the 50 sample loss is 0.979298, accuracy is 0.880000 

 401 the iteration and the 50 sample loss is 3.482923, accuracy is 0.840000 
 402 the iteration and the 50 sample loss is 2.179819, accuracy is 0.900000 
 403 the iteration and the 50 sample loss is 5.222996, accuracy is 0.780000 
 404 the iteration and the 50 sample loss is 2.812206, accuracy is 0.740000 
 405 the iteration and the 50 sample loss is 2.006791, accuracy is 0.840000 
 406 the iteration and the 50 sample loss is 2.300585, accuracy is 0.820000 
 407 the iteration and the 50 sample loss is 2.776532, accuracy is 0.860000 
 408 the iteration and the 50 sample loss is 1.498939, accuracy is 0.880000 
 409 the iteration and the 50 sample loss is 1.671275, accuracy is 0.920000 
 410 the iteration and the 50 sample loss is 2.497543, accuracy is 0.880000 
 411 the iteration and the 50 sample loss is 3.824326, accuracy is 0.860000 
 412 the iteration and the 50 sample loss is 1.373718, accuracy is 0.920000 
 413 the iteration and the 50 sample loss is 4.872751, accuracy is 0.820000 

 601 the iteration and the 50 sample loss is 1.578820, accuracy is 0.820000 
 602 the iteration and the 50 sample loss is 5.969700, accuracy is 0.840000 
 603 the iteration and the 50 sample loss is 1.370728, accuracy is 0.840000 
 604 the iteration and the 50 sample loss is 1.322069, accuracy is 0.880000 
 605 the iteration and the 50 sample loss is 0.804450, accuracy is 0.920000 
 606 the iteration and the 50 sample loss is 2.104170, accuracy is 0.840000 
 607 the iteration and the 50 sample loss is 1.322067, accuracy is 0.860000 
 608 the iteration and the 50 sample loss is 3.166091, accuracy is 0.760000 
 609 the iteration and the 50 sample loss is 2.178587, accuracy is 0.780000 
 610 the iteration and the 50 sample loss is 2.100613, accuracy is 0.800000 
 611 the iteration and the 50 sample loss is 0.277321, accuracy is 0.900000 
 612 the iteration and the 50 sample loss is 3.265896, accuracy is 0.760000 
 613 the iteration and the 50 sample loss is 2.079399, accuracy is 0.780000 

 801 the iteration and the 50 sample loss is 2.601589, accuracy is 0.760000 
 802 the iteration and the 50 sample loss is 0.720325, accuracy is 0.880000 
 803 the iteration and the 50 sample loss is 0.706386, accuracy is 0.900000 
 804 the iteration and the 50 sample loss is 0.821824, accuracy is 0.860000 
 805 the iteration and the 50 sample loss is 1.368086, accuracy is 0.800000 
 806 the iteration and the 50 sample loss is 1.425912, accuracy is 0.920000 
 807 the iteration and the 50 sample loss is 1.995242, accuracy is 0.880000 
 808 the iteration and the 50 sample loss is 0.723690, accuracy is 0.880000 
 809 the iteration and the 50 sample loss is 3.163245, accuracy is 0.840000 
 810 the iteration and the 50 sample loss is 1.578324, accuracy is 0.860000 
 811 the iteration and the 50 sample loss is 0.597783, accuracy is 0.880000 
 812 the iteration and the 50 sample loss is 1.083995, accuracy is 0.880000 
 813 the iteration and the 50 sample loss is 1.959144, accuracy is 0.800000 

...

 1201 the iteration and the 50 sample loss is 0.706047, accuracy is 0.980000 
 1202 the iteration and the 50 sample loss is 1.617521, accuracy is 0.840000 
 1203 the iteration and the 50 sample loss is 1.058480, accuracy is 0.860000 
 1204 the iteration and the 50 sample loss is 1.789384, accuracy is 0.860000 
 1205 the iteration and the 50 sample loss is 1.731802, accuracy is 0.860000 
 1206 the iteration and the 50 sample loss is 0.255575, accuracy is 0.940000 
 1207 the iteration and the 50 sample loss is 0.298454, accuracy is 0.920000 
 1208 the iteration and the 50 sample loss is 0.808378, accuracy is 0.840000 
 1209 the iteration and the 50 sample loss is 1.709034, accuracy is 0.860000 
 1210 the iteration and the 50 sample loss is 0.992628, accuracy is 0.920000 
 1211 the iteration and the 50 sample loss is 1.413462, accuracy is 0.860000 
 1212 the iteration and the 50 sample loss is 2.351537, accuracy is 0.780000 
 1213 the iteration and the 50 sample loss is 1.247551, accuracy

 1401 the iteration and the 50 sample loss is 0.616933, accuracy is 0.880000 
 1402 the iteration and the 50 sample loss is 0.676523, accuracy is 0.880000 
 1403 the iteration and the 50 sample loss is 0.773155, accuracy is 0.880000 
 1404 the iteration and the 50 sample loss is 1.887180, accuracy is 0.840000 
 1405 the iteration and the 50 sample loss is 1.495489, accuracy is 0.860000 
 1406 the iteration and the 50 sample loss is 0.747053, accuracy is 0.900000 
 1407 the iteration and the 50 sample loss is 0.894519, accuracy is 0.880000 
 1408 the iteration and the 50 sample loss is 1.571334, accuracy is 0.800000 
 1409 the iteration and the 50 sample loss is 0.159128, accuracy is 0.940000 
 1410 the iteration and the 50 sample loss is 0.518892, accuracy is 0.940000 
 1411 the iteration and the 50 sample loss is 0.169508, accuracy is 0.900000 
 1412 the iteration and the 50 sample loss is 0.430732, accuracy is 0.940000 
 1413 the iteration and the 50 sample loss is 0.217712, accuracy

 1601 the iteration and the 50 sample loss is 1.065945, accuracy is 0.880000 
 1602 the iteration and the 50 sample loss is 0.552295, accuracy is 0.940000 
 1603 the iteration and the 50 sample loss is 1.976819, accuracy is 0.760000 
 1604 the iteration and the 50 sample loss is 1.082983, accuracy is 0.820000 
 1605 the iteration and the 50 sample loss is 0.509718, accuracy is 0.900000 
 1606 the iteration and the 50 sample loss is 0.692828, accuracy is 0.920000 
 1607 the iteration and the 50 sample loss is 0.829505, accuracy is 0.880000 
 1608 the iteration and the 50 sample loss is 0.850249, accuracy is 0.880000 
 1609 the iteration and the 50 sample loss is 1.213698, accuracy is 0.840000 
 1610 the iteration and the 50 sample loss is 1.155422, accuracy is 0.880000 
 1611 the iteration and the 50 sample loss is 1.288835, accuracy is 0.880000 
 1612 the iteration and the 50 sample loss is 0.688450, accuracy is 0.880000 
 1613 the iteration and the 50 sample loss is 1.852568, accuracy

 1801 the iteration and the 50 sample loss is 0.767068, accuracy is 0.860000 
 1802 the iteration and the 50 sample loss is 2.353549, accuracy is 0.840000 
 1803 the iteration and the 50 sample loss is 1.053538, accuracy is 0.860000 
 1804 the iteration and the 50 sample loss is 0.799613, accuracy is 0.860000 
 1805 the iteration and the 50 sample loss is 0.469004, accuracy is 0.880000 
 1806 the iteration and the 50 sample loss is 1.353787, accuracy is 0.760000 
 1807 the iteration and the 50 sample loss is 0.813979, accuracy is 0.840000 
 1808 the iteration and the 50 sample loss is 1.436694, accuracy is 0.820000 
 1809 the iteration and the 50 sample loss is 1.124085, accuracy is 0.740000 
 1810 the iteration and the 50 sample loss is 1.315370, accuracy is 0.780000 
 1811 the iteration and the 50 sample loss is 0.175635, accuracy is 0.940000 
 1812 the iteration and the 50 sample loss is 1.019223, accuracy is 0.840000 
 1813 the iteration and the 50 sample loss is 1.177371, accuracy

 the d1ample, real pic number is 7, neural network predicted number is 7
 the d2ample, real pic number is 2, neural network predicted number is 2
 the d3ample, real pic number is 1, neural network predicted number is 1
 the d4ample, real pic number is 0, neural network predicted number is 0
 the d5ample, real pic number is 4, neural network predicted number is 4
 the d6ample, real pic number is 1, neural network predicted number is 1
 the d7ample, real pic number is 4, neural network predicted number is 4
 the d8ample, real pic number is 9, neural network predicted number is 9
 the d9ample, real pic number is 5, neural network predicted number is 6
 the d10ample, real pic number is 9, neural network predicted number is 9
 the d11ample, real pic number is 0, neural network predicted number is 0
 the d12ample, real pic number is 6, neural network predicted number is 6
 the d13ample, real pic number is 9, neural network predicted number is 9
 the d14ample, real pic number is 0, neural net